# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'ionut-automl-run'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-143935
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-143935


In [3]:
# Check if there is a compute cluster, if yes use it
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "ionut-compute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Create TabularDataset using TabularDatasetFactory
from azureml.core.dataset import Dataset

# Try to load the dataset from the Workspace. Otherwise, create it from the file

found = False
key = "Heart Disease Dataset"
description_text = "Heart Disease DataSet for Udacity Capstone Project (Source: UCI)"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/ionut-ml/nd00333-capstone/main/starter_file/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

# Split data into train and test sets.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Concat X_train & y_train into one df
import pandas as pd
df_train = pd.concat([x_train, y_train], axis=1)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# Save training data as csv

if not os.path.isdir("train_data"):
    os.mkdir("train_data")

df_train.to_csv("./train_data/train_data.csv",index=False)

In [7]:
# Upload data as tabular dataset

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='train_data', target_path='data')

# create a dataset referencing the cloud location
train_ds = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/train_data.csv'))])


Uploading an estimated of 1 files
Uploading train_data/train_data.csv
Uploaded train_data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [11]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 5,
    "max_cores_per_iteration": -1,
#     "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=cpu_cluster_name,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_ds,
    label_column_name='target',
    n_cross_validations=5,
    featurization= 'auto',
    enable_onnx_compatible_models=True,
    debug_log = "automl_errors.log",
    **automl_settings
)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ionut-compute with default configuration
Running on remote compute: ionut-compute


Experiment,Id,Type,Status,Details Page,Docs Page
ionut-automl-run,AutoML_a5ab2c31-5a1f-4810-9cc8-e92c84bebaa2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Get best automl run
best_automl_run, best_automl_onnx_model = automl_run.get_output(return_onnx_model=True)

In [ ]:
#TODO: Save the best model
from azureml.automl.runtime.onnx_convert import OnnxConverter

# Save model using Onnx Converter
OnnxConverter.save_onnx_model(best_automl_onnx_model, file_path='./outputs/best_automl_onnx_model.onnx')

In [ ]:
# Get the hyperparameters for the best run
best_automl_run, best_automl_onnx_model = automl_run.get_output()
best_automl_run.get_tags()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service